## Preprocessing

### NOTE: Please run this notebook using Google Colab!

In [22]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Check to see the columns were dropped:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

### Binning "Application Type" Data

In [4]:
# Look at APPLICATION_TYPE value counts for binning
counts_apptype = application_df["APPLICATION_TYPE"].value_counts()

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
app_types_to_replace = list(counts_apptype[counts_apptype<500].index)

# Replace in dataframe
for app in app_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
counts_classif = application_df["CLASSIFICATION"].value_counts()
counts_classif

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts_classif.loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
classif_to_replace = list(counts_classif[counts_classif<1000].index)

# Replace in dataframe
for cls in classif_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

## Data Preprocessing

In [9]:
# Determine which columns are categorical vs numerical
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df = pd.get_dummies(application_df)
app_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = app_dummies_df.drop("IS_SUCCESSFUL",axis=1).values
y = app_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the number of features in the dataset:
num_features = len(X_train[0])
num_features

43

## Compile, Train and Evaluate the Model

Attempting to manually write a neural network model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and 
# hidden nodes for each layer.
num_features = len(X_train[0])
nodes_layer1 = 30
nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_layer1,
                             input_dim=num_features,
                             activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_layer2,
                             activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units = 1,
                             activation = "sigmoid")
)

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_acc = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_acc}")

Now using Keras Tuner to find a more optimized model.

In [14]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.3 MB/s eta 0:00:00


In [15]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
  nn = tf.keras.models.Sequential()
  
  # Allow kt to decide which activation function to use
  activation = hp.Choice('activation', ['relu','tanh','sigmoid'])

  # and how many neurons will be in the first layer
  nn.add(tf.keras.layers.Dense(units = hp.Int('first_units',
                                            min_value = 1,
                                            max_value = 40,
                                            step = 5), 
                               activation = activation, 
                               input_dim = num_features))
  
  # and how many more hidden layers are needed, and how many neurons per layer
  for i in range(hp.Int('num_layers', 1, 6)):
    nn.add(tf.keras.layers.Dense(units = hp.Int('units_' + str(i),
                                                min_value = 1,
                                                max_value = 40,
                                                step = 5), 
                                 activation = activation))
  
  # and code the output layer
  nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

  # and compile the model
  nn.compile(loss = "binary_crossentropy",
             optimizer = 'adam',
             metrics = ["accuracy"])
  
  return nn

In [16]:
# Import Keras Tuner
import keras_tuner as kt

# Create a HyperBand tuner instance
tuner = kt.Hyperband(
    create_model,
    objective = 'val_accuracy',
    max_epochs = 50,
    hyperband_iterations = 2
)

In [17]:
# Run the tuner
tuner.search(X_train_scaled, y_train, 
             epochs = 50,
             validation_data = (X_test_scaled, y_test))

Trial 180 Complete [00h 02m 35s]
val_accuracy: 0.7315452098846436

Best val_accuracy So Far: 0.734810471534729
Total elapsed time: 01h 47m 31s


In [21]:
# Get the best model's hyperparameters and evaluation stats
best_hyper = tuner.get_best_hyperparameters()[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 36,
 'num_layers': 2,
 'units_0': 16,
 'units_1': 6,
 'units_2': 1,
 'units_3': 21,
 'units_4': 36,
 'units_5': 11,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0075'}

In [19]:
# Evaluate the best model against test data
top_model = tuner.get_best_models()[0]
model_loss, model_acc = top_model.evaluate(X_test_scaled, y_test, verbose=2)

268/268 - 1s - loss: 0.5544 - accuracy: 0.7348 - 703ms/epoch - 3ms/step


In [20]:
# Export our model to HDF5 file
top_model.save('AlphabetSoupCharity_VentureFunder.h5')